# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあればinstall.packages()で適宜インストールしてください
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
require("RPostgreSQL")
require("tidyr")
require("dplyr")
require("stringr")
require("caret")
require("lubridate")
require("rsample")
require("recipes")
require("themis")

host <- "db"
port <- Sys.getenv()["PG_PORT"]
dbname <- Sys.getenv()["PG_DATABASE"]
user <- Sys.getenv()["PG_USER"]
password <- Sys.getenv()["PG_PASSWORD"]

con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
df_customer <- dbGetQuery(con,"SELECT * FROM customer")
df_category <- dbGetQuery(con,"SELECT * FROM category")
df_product <- dbGetQuery(con,"SELECT * FROM product")
df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
df_store <- dbGetQuery(con,"SELECT * FROM store")
df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: tidyr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stringr

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: rsample

Loading required package: recipes


Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


Loading required package: themis



# 演習問題

---
> R-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt %>% head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> R-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [3]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% head(10)

,sales_ymd,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [4]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% head(10) %>% rename("sales_date" = sales_ymd)

,sales_date,customer_id,product_cd,amount
,<int>,<chr>,<chr>,<int>
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> R-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [5]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% filter(customer_id == "CS018205000001")

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [6]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% filter(customer_id == "CS018205000001" & amount >= 1000) 

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> R-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
df_receipt %>%
    select(sales_ymd, customer_id, product_cd, quantity, amount) %>%
    filter(customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 5))

sales_ymd,customer_id,product_cd,quantity,amount
<int>,<chr>,<chr>,<int>,<int>
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> R-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [8]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% filter(customer_id == "CS018205000001" & 1000 <= amount & amount <= 2000) 

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401005,1100


---
> R-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [9]:
df_receipt %>% select(sales_ymd, customer_id, product_cd, amount) %>% filter(customer_id == "CS018205000001" & product_cd != "P071401019") 

sales_ymd,customer_id,product_cd,amount
<int>,<chr>,<chr>,<int>
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200


---
> R-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store %>% filter(!(prefecture_cd == "13" | floor_area > 900))`

In [10]:
df_store %>% filter(prefecture_cd != "13" & floor_area <= 900)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808


---
> R-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [11]:
df_store %>% filter(str_detect(store_cd, "^S14")) %>% head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838


---
> R-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [12]:
df_customer %>% filter(str_detect(customer_id, "1$")) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> R-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [13]:
df_store %>% filter(str_detect(address, "横浜市"))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657


---
> R-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [14]:
df_customer %>% filter(str_detect(status_cd, "^[A-F]")) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> R-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [15]:
df_customer %>% filter(str_detect(status_cd, "[1-9]$")) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> R-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [16]:
df_customer %>% filter(str_detect(status_cd, "^[A-F]") & str_detect(status_cd, "[1-9]$")) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> R-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [17]:
df_store %>% filter(str_detect(tel_no, "[0-9]{3}-[0-9]{3}-[0-9]{4}"))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574


---
> R-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [18]:
df_customer %>% arrange(birth_day) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> R-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [19]:
df_customer %>% arrange(desc(birth_day)) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> R-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [130]:
df_receipt %>% select(customer_id, amount) %>% arrange(desc(amount)) %>% mutate(rank = min_rank(desc(amount))) %>% head(10)

,customer_id,amount,rank
,<chr>,<int>,<int>
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,7
8,CS021515000089,5440,7
9,CS015515000083,5280,9


---
> R-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [21]:
df_receipt %>% select(customer_id, amount) %>% arrange(desc(amount)) %>% mutate(rank = row_number(desc(amount))) %>% head(10)

,customer_id,amount,rank
,<chr>,<int>,<int>
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9


---
> R-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [22]:
df_receipt %>% count() %>% pull()

[1] 104681

---
> R-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [136]:
df_receipt %>% distinct(customer_id) %>% nrow()

[1] 8307

---
> R-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [137]:
df_receipt %>% group_by(store_cd) %>% summarise(amount = sum(amount), quantity = sum(quantity))

store_cd,amount,quantity
<chr>,<int>,<int>
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


---
> R-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [138]:
df_receipt %>% group_by(customer_id) %>% summarise(max_ymd = max(sales_ymd)) %>% head(10)

customer_id,max_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20190731
CS001115000010,20190405
CS001205000004,20190625
CS001205000006,20190224
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
> R-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [139]:
df_receipt %>% group_by(customer_id) %>% summarise(min_ymd = min(sales_ymd)) %>% head(10)

customer_id,min_ymd
<chr>,<int>
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811


---
## R-026 PASS
> R-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [140]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd = max(sales_ymd), min_ymd = min(sales_ymd)) %>%
    filter(max_ymd != min_ymd) %>%
    head(10)

customer_id,max_ymd,min_ymd
<chr>,<int>,<int>
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


---
> R-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [28]:
df_receipt %>% group_by(store_cd) %>% summarize(Mean = mean(amount)) %>% arrange(desc(Mean)) %>% head(5)

store_cd,Mean
<chr>,<dbl>
S13052,402.8675
S13015,351.1120
S13003,350.9155
S14010,348.7913
S13001,348.4704


---
> R-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [29]:
df_receipt %>% group_by(store_cd) %>% summarize(Median = median(amount)) %>% arrange(desc(Median)) %>% head(5)

store_cd,Median
<chr>,<dbl>
S13052,190
S14010,188
S14050,185
S13003,180
S13018,180


---
> R-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [30]:
df_receipt %>% group_by(store_cd) %>% summarize(Mode = names(which.max(table(product_cd)))) %>% head(10)

store_cd,Mode
<chr>,<chr>
S12007,P060303001
S12013,P060303001
S12014,P060303001
S12029,P060303001
S12030,P060303001
S13001,P060303001
S13002,P060303001
S13003,P071401001
S13004,P060303001


---
> R-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [31]:
df_receipt %>% group_by(store_cd) %>% summarize(Var = var(amount)) %>% arrange(desc(Var)) %>% head(5)

store_cd,Var
<chr>,<dbl>
S13052,441863.3
S14011,306442.2
S14034,297068.4
S13001,295558.8
S13015,295427.2


---
> R-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [32]:
df_receipt %>% group_by(store_cd) %>% summarize(Sd = sd(amount)) %>% arrange(desc(Sd)) %>% head(5)

store_cd,Sd
<chr>,<dbl>
S13052,664.7280
S14011,553.5723
S14034,545.0398
S13001,543.6532
S13015,543.5321


---
> R-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [33]:
df_receipt$amount %>% quantile()

0%   25%   50%   75%  100% 
   10   102   170   288 10925

---
> R-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [34]:
df_receipt %>% group_by(store_cd) %>% summarize(Mean = mean(amount)) %>% filter(Mean >= 330)

store_cd,Mean
<chr>,<dbl>
S12013,330.1941
S13001,348.4704
S13003,350.9155
S13004,330.9439
S13015,351.1120
S13019,330.2086
S13020,337.8799
S13052,402.8675
S14010,348.7913


---
> R-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [35]:
df_receipt %>% group_by(customer_id) %>% filter(!str_detect(customer_id, "^Z")) %>% summarize(Sum = sum(amount)) %>% summarize(mean(Sum)) %>% pull()

[1] 2547.742

---
> R-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [36]:
Mean <- df_receipt %>% group_by(customer_id) %>% filter(!str_detect(customer_id, "^Z")) %>% summarize(Sum = sum(amount)) %>% summarize(mean(Sum)) %>% pull()
# Mean = 2547.74223452926
df_receipt %>% group_by(customer_id) %>% filter(!str_detect(customer_id, "^Z")) %>% summarize(Sum = sum(amount)) %>% filter(Sum >= Mean) %>% head(10)

customer_id,Sum
<chr>,<int>
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639
CS001215000040,3496
CS001304000006,3726
CS001305000005,3485
CS001305000011,4370


---
> R-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [37]:
left_join(df_receipt, df_store, by="store_cd") %>% select(everything(df_receipt), store_name) %>% head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店


---
> R-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [38]:
left_join(df_product, df_category, by="category_small_cd") %>% select(everything(df_product), category_small_name) %>% head(10)

,product_cd,category_major_cd.x,category_medium_cd.x,category_small_cd,unit_price,unit_cost,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
1,P040101001,04,0401,040101,198,149,弁当類
2,P040101002,04,0401,040101,218,164,弁当類
3,P040101003,04,0401,040101,230,173,弁当類
4,P040101004,04,0401,040101,248,186,弁当類
5,P040101005,04,0401,040101,268,201,弁当類
6,P040101006,04,0401,040101,298,224,弁当類
7,P040101007,04,0401,040101,338,254,弁当類
8,P040101008,04,0401,040101,420,315,弁当類
9,P040101009,04,0401,040101,498,374,弁当類


---
> R-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [39]:
left_join(df_customer, df_receipt, by="customer_id") %>%
    filter(gender_cd == 1 & !str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Sum = replace_na(Sum, 0)) %>%
    head(10)

customer_id,Sum
<chr>,<int>
CS001112000009,0
CS001112000019,0
CS001112000021,0
CS001112000023,0
CS001112000024,0
CS001112000029,0
CS001112000030,0
CS001113000004,1298
CS001113000010,0


---
> R-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [148]:
# df_receipt %>% group_by(sales_ymd) %>% count(customer_id) %>% head(20)
a <- df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(come_days = n_distinct(sales_ymd)) %>%
    arrange(desc(come_days)) %>%
    head(20)

b <- df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    arrange(desc(Sum)) %>%
    head(20)

full_join(a, b, by = "customer_id")

customer_id,come_days,Sum
<chr>,<int>,<int>
CS040214000008,23,NA
CS010214000010,22,18585
CS015415000185,22,20153
CS010214000002,21,NA
CS028415000007,21,19127
CS016415000141,20,18372
CS017415000097,20,23086
CS014214000023,19,NA
CS021514000045,19,NA


---
## R-040 PASS
> R-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [150]:
df_store_tmp <- df_store

df_product_tmp <- df_product

df_store_tmp["key"] <- 0

df_product_tmp["key"] <- 0

full_join(df_store_tmp, df_product_tmp, by = "key", relationship = "many-to-many") %>% nrow()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area,key,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101001,04,0401,040101,198,149
2,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101002,04,0401,040101,218,164
3,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101003,04,0401,040101,230,173
4,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101004,04,0401,040101,248,186
5,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101005,04,0401,040101,268,201
6,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698,0,P040101006,04,0401,040101,298,224


---
> R-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [155]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(sales_ymd_lagged = lag(sales_ymd, n = 1), Sum_lagged = lag(Sum, n = 1), diff_amount = Sum - Sum_lagged) %>%
    head(10)

sales_ymd,Sum,sales_ymd_lagged,Sum_lagged,diff_amount
<int>,<int>,<int>,<int>,<int>
20170101,33723,NA,NA,NA
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981


---
> R-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [157]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(one_day_before = lag(Sum, n = 1)) %>%
    mutate(two_day_before = lag(Sum, n = 2)) %>%
    mutate(three_day_before = lag(Sum, n = 3)) %>%
    drop_na(everything()) %>%
    head(10)

sales_ymd,Sum,one_day_before,two_day_before,three_day_before
<int>,<int>,<int>,<int>,<int>
20170104,36165,27503,24165,33723
20170105,37830,36165,27503,24165
20170106,32387,37830,36165,27503
20170107,23415,32387,37830,36165
20170108,24737,23415,32387,37830
20170109,26718,24737,23415,32387
20170110,20143,26718,24737,23415
20170111,24287,20143,26718,24737
20170112,23526,24287,20143,26718


---
> R-043: レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [163]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    mutate(agec = case_when(
            10 <= age & age < 20 ~ "10-19",
            20 <= age & age < 30 ~ "20-29",
            30 <= age & age < 40 ~ "30-39",
            40 <= age & age < 50 ~ "40-49",
            50 <= age & age < 60 ~ "50-59",
            60 <= age & age < 70 ~ "60-69",
            70 <= age & age < 80 ~ "70-79",
            80 <= age & age < 90 ~ "80-89",
            90 <= age & age < 100 ~ "90-99",)
           ) %>%
    group_by(gender_cd, agec, .groups = "drop") %>%
    summarize(Sum = sum(amount), .groups = "drop") %>%
    pivot_wider(names_from = "gender_cd", values_from = "Sum") %>%
    select(-`.groups`, -`NA`) %>%
    rename(male = "0", female = "1", unknown = "9") %>%
    filter(agec != "NA")

agec,male,female,unknown
<chr>,<int>,<int>,<int>
10-19,1591,149836,4317
20-29,72940,1363724,44328
30-39,177322,693047,50441
40-49,19355,9320791,483512
50-59,54320,6685192,342923
60-69,272469,987741,71418
70-79,13435,29764,2427
80-89,46360,262923,5111
90-99,NA,6260,NA


---
> R-044: 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [164]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    mutate(agec = case_when(
            10 <= age & age < 20 ~ "10-19",
            20 <= age & age < 30 ~ "20-29",
            30 <= age & age < 40 ~ "30-39",
            40 <= age & age < 50 ~ "40-49",
            50 <= age & age < 60 ~ "50-59",
            60 <= age & age < 70 ~ "60-69",
            70 <= age & age < 80 ~ "70-79",
            80 <= age & age < 90 ~ "80-89",
            90 <= age & age < 100 ~ "90-99",)
           ) %>%
    group_by(gender_cd, agec) %>%
    summarize(Sum = sum(amount), .groups = "drop") %>%
    mutate(gender_cd = case_when(
            gender_cd == "0" ~ "00",
            gender_cd == "1" ~ "01",
            gender_cd == "9" ~ "99")
           )

gender_cd,agec,Sum
<chr>,<chr>,<int>
00,10-19,1591
00,20-29,72940
00,30-39,177322
00,40-49,19355
00,50-59,54320
00,60-69,272469
00,70-79,13435
00,80-89,46360
01,10-19,149836


---
> R-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [46]:
df_customer %>% mutate(birth_day = format(birth_day, "%Y%m%d")) %>% select(customer_id, birth_day) %>% head(10)

,customer_id,birth_day
,<chr>,<chr>
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


---
> R-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [47]:
df_customer %>% mutate(application_date = as.Date(application_date, format = "%Y%m%d")) %>% select(customer_id, application_date) %>% head(10)

,customer_id,application_date
,<chr>,<date>
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


---
> R-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [48]:
df_receipt %>% mutate(sales_ymd = as_date(format(sales_ymd), format = "%Y%m%d")) %>% select(receipt_no, receipt_sub_no, sales_ymd) %>% head(10)

,receipt_no,receipt_sub_no,sales_ymd
,<int>,<int>,<date>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [49]:
df_receipt %>% mutate(sales_epoch = as_date(as.POSIXct(sales_epoch))) %>% select(receipt_no, receipt_sub_no, sales_epoch) %>% head(10)

,receipt_no,receipt_sub_no,sales_epoch
,<int>,<int>,<date>
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> R-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [50]:
df_receipt %>% mutate(sales_epoch = year(as_date(as.POSIXct(sales_epoch)))) %>% select(receipt_no, receipt_sub_no, sales_epoch) %>% head(10)

,receipt_no,receipt_sub_no,sales_epoch
,<int>,<int>,<dbl>
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


---
> R-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [51]:
df_receipt %>% mutate(sales_epoch = format.Date(as_date(as.POSIXct(sales_epoch)), "%m")) %>% select(receipt_no, receipt_sub_no, sales_epoch) %>% head(10)

,receipt_no,receipt_sub_no,sales_epoch
,<int>,<int>,<chr>
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05


---
> R-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [52]:
df_receipt %>% mutate(sales_epoch = format.Date(as_date(as.POSIXct(sales_epoch)), "%d")) %>% select(receipt_no, receipt_sub_no, sales_epoch) %>% head(10)

,receipt_no,receipt_sub_no,sales_epoch
,<int>,<int>,<chr>
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


---
> R-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [53]:
df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(over2000 = case_when(
        Sum <= 2000 ~ 0,
        TRUE ~ 1)
           ) %>%
    select(customer_id, Sum, over2000) %>%
    head(10)

customer_id,Sum,over2000
<chr>,<int>,<dbl>
CS001113000004,1298,0
CS001114000005,626,0
CS001115000010,3044,1
CS001205000004,1988,0
CS001205000006,3337,1
CS001211000025,456,0
CS001212000027,448,0
CS001212000031,296,0
CS001212000046,228,0


---
> R-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [54]:
df_customer %>%
    mutate(is_Tokyo = case_when(
        str_detect(postal_cd, "^1|^20[0-9]") ~ 1,
        TRUE ~ 0)
           ) %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(is_Tokyo) %>%
    distinct(customer_id) %>%
    count()

is_Tokyo,n
<dbl>,<int>
0,3906
1,4400


---
> R-054:  顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [55]:
df_customer %>%
    mutate(pref = case_when(
        str_detect(address, "埼玉県") ~ 11,
        str_detect(address, "千葉県") ~ 12,
        str_detect(address, "東京都") ~ 13,
        str_detect(address, "神奈川県") ~ 14)
           ) %>%
    select(customer_id, address, pref) %>%
    head(10)

,customer_id,address,pref
,<chr>,<chr>,<dbl>
1,CS021313000114,神奈川県伊勢原市粟窪**********,14
2,CS037613000071,東京都江東区南砂**********,13
3,CS031415000172,東京都渋谷区代々木**********,13
4,CS028811000001,神奈川県横浜市泉区和泉町**********,14
5,CS001215000145,東京都大田区仲六郷**********,13
6,CS020401000016,東京都板橋区若木**********,13
7,CS015414000103,東京都江東区北砂**********,13
8,CS029403000008,千葉県浦安市海楽**********,12
9,CS015804000004,東京都江東区北砂**********,13


---
> R-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [56]:
quantiles <- df_receipt %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    summarize(quantiles = quantile(Sum)) %>%
    pull()
quantiles
df_receipt %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Category = case_when(
        quantiles["0%"] <= Sum & Sum < quantiles["25%"] ~ 1,
        quantiles["25%"] <= Sum & Sum < quantiles["50%"] ~ 2,
        quantiles["50%"] <= Sum & Sum < quantiles["75%"] ~ 3,
        TRUE ~ 4)
        ) %>%
    select(customer_id, Sum, Category) %>%
    head(10)

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.”


0%        25%        50%        75%       100% 
      70.0      548.5     1478.0     3651.0 12395003.0

customer_id,Sum,Category
<chr>,<int>,<dbl>
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1
CS001212000027,448,1
CS001212000031,296,1
CS001212000046,228,1


---
> R-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [165]:
df_customer %>%
    mutate(agec = case_when(
            10 <= age & age < 20 ~ "10-19",
            20 <= age & age < 30 ~ "20-29",
            30 <= age & age < 40 ~ "30-39",
            40 <= age & age < 50 ~ "40-49",
            50 <= age & age < 60 ~ "50-59",
            60 <= age ~ "60-")
           ) %>%
    select(customer_id, birth_day, age, agec) %>%
    head(10)

,customer_id,birth_day,age,agec
,<chr>,<date>,<int>,<chr>
1,CS021313000114,1981-04-29,37,30-39
2,CS037613000071,1952-04-01,66,60-
3,CS031415000172,1976-10-04,42,40-49
4,CS028811000001,1933-03-27,86,60-
5,CS001215000145,1995-03-29,24,20-29
6,CS020401000016,1974-09-15,44,40-49
7,CS015414000103,1977-08-09,41,40-49
8,CS029403000008,1973-08-17,45,40-49
9,CS015804000004,1931-05-02,87,60-


---
> R-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [58]:
df_customer %>%
    mutate(agec = case_when(
            10 <= age & age < 20 ~ "10-19",
            20 <= age & age < 30 ~ "20-29",
            30 <= age & age < 40 ~ "30-39",
            40 <= age & age < 50 ~ "40-49",
            50 <= age & age < 60 ~ "50-59",
            60 <= age ~ "60-")
           ) %>%
    mutate(gender_agec = case_when(
        agec == "10-19" & gender_cd == 0 ~ "10-19m",
        agec == "10-19" & gender_cd == 1 ~ "10-19f",
        agec == "10-19" & gender_cd == 9 ~ "10-19na",
        agec == "20-29" & gender_cd == 0 ~ "20-29m",
        agec == "20-29" & gender_cd == 1 ~ "20-29f",
        agec == "20-29" & gender_cd == 9 ~ "20-29na",
        agec == "30-39" & gender_cd == 0 ~ "30-39m",
        agec == "30-39" & gender_cd == 1 ~ "30-39f",
        agec == "30-39" & gender_cd == 9 ~ "30-39na",
        agec == "40-49" & gender_cd == 0 ~ "40-49m",
        agec == "40-49" & gender_cd == 1 ~ "40-49f",
        agec == "40-49" & gender_cd == 9 ~ "40-49na",
        agec == "50-59" & gender_cd == 0 ~ "50-59m",
        agec == "50-59" & gender_cd == 1 ~ "50-59f",
        agec == "50-59" & gender_cd == 9 ~ "50-59na",
        agec == "60-" & gender_cd == 0 ~ "60-m",
        agec == "60-" & gender_cd == 1 ~ "60-f",
        agec == "60-" & gender_cd == 9 ~ "60-na",
    )) %>%
    select(customer_id, birth_day, agec, gender_agec) %>%
    head(10)

,customer_id,birth_day,agec,gender_agec
,<chr>,<date>,<chr>,<chr>
1,CS021313000114,1981-04-29,30-39,30-39f
2,CS037613000071,1952-04-01,60-,60-na
3,CS031415000172,1976-10-04,40-49,40-49f
4,CS028811000001,1933-03-27,60-,60-f
5,CS001215000145,1995-03-29,20-29,20-29f
6,CS020401000016,1974-09-15,40-49,40-49m
7,CS015414000103,1977-08-09,40-49,40-49f
8,CS029403000008,1973-08-17,40-49,40-49m
9,CS015804000004,1931-05-02,60-,60-m


---
> R-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [169]:
df_customer %>%
    mutate(gender_cd0 = ifelse(gender_cd == 0, 1, 0)) %>%
    mutate(gender_cd1 = ifelse(gender_cd == 1, 1, 0)) %>%
    mutate(gender_cd9 = ifelse(gender_cd == 9, 1, 0)) %>%
    select(customer_id, gender_cd0, gender_cd1, gender_cd9) %>%
    head(10)

,customer_id,gender_cd0,gender_cd1,gender_cd9
,<chr>,<dbl>,<dbl>,<dbl>
1,CS021313000114,0,1,0
2,CS037613000071,0,0,1
3,CS031415000172,0,1,0
4,CS028811000001,0,1,0
5,CS001215000145,0,1,0
6,CS020401000016,1,0,0
7,CS015414000103,0,1,0
8,CS029403000008,1,0,0
9,CS015804000004,1,0,0


---
> R-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [60]:
df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Sum_std = (Sum - mean(Sum))/sd(Sum)) %>%
    head(10)

customer_id,Sum,Sum_std
<chr>,<int>,<dbl>
CS001113000004,1298,-0.4593502
CS001114000005,626,-0.7063478
CS001115000010,3044,0.1824025
CS001205000004,1988,-0.2057366
CS001205000006,3337,0.2900964
CS001211000025,456,-0.7688324
CS001212000027,448,-0.7717728
CS001212000031,296,-0.8276413
CS001212000046,228,-0.8526351


---
> R-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [61]:
df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Sum_pt = (Sum - min(Sum))/(max(Sum)-min(Sum))) %>%
    head(10)

customer_id,Sum,Sum_pt
<chr>,<int>,<dbl>
CS001113000004,1298,0.053354188
CS001114000005,626,0.024157108
CS001115000010,3044,0.129214460
CS001205000004,1988,0.083333333
CS001205000006,3337,0.141944734
CS001211000025,456,0.016770942
CS001212000027,448,0.016423358
CS001212000031,296,0.009819256
CS001212000046,228,0.006864790


---
> R-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [62]:
df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Sum_log10 = log(Sum, 10)) %>%
    head(10)

customer_id,Sum,Sum_log10
<chr>,<int>,<dbl>
CS001113000004,1298,3.113275
CS001114000005,626,2.796574
CS001115000010,3044,3.483445
CS001205000004,1988,3.298416
CS001205000006,3337,3.523356
CS001211000025,456,2.658965
CS001212000027,448,2.651278
CS001212000031,296,2.471292
CS001212000046,228,2.357935


---
> R-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [170]:
df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    mutate(Sum_log10 = log(Sum + 0.5)) %>%
    head(10)

customer_id,Sum,Sum_log10
<chr>,<int>,<dbl>
CS001113000004,1298,7.168965
CS001114000005,626,6.440149
CS001115000010,3044,8.021092
CS001205000004,1988,7.595136
CS001205000006,3337,8.112977
CS001211000025,456,6.123589
CS001212000027,448,6.105909
CS001212000031,296,5.692047
CS001212000046,228,5.431536


---
> R-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [64]:
df_product %>% mutate(profit = unit_price - unit_cost) %>% head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
1,P040101001,04,0401,040101,198,149,49
2,P040101002,04,0401,040101,218,164,54
3,P040101003,04,0401,040101,230,173,57
4,P040101004,04,0401,040101,248,186,62
5,P040101005,04,0401,040101,268,201,67
6,P040101006,04,0401,040101,298,224,74
7,P040101007,04,0401,040101,338,254,84
8,P040101008,04,0401,040101,420,315,105
9,P040101009,04,0401,040101,498,374,124


---
> R-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [171]:
df_product %>% mutate(profit_percentage = (unit_price - unit_cost)/unit_price) %>% summarize(mean(profit_percentage, na.rm=TRUE)) %>% pull()

[1] 0.2491139

---
> R-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [175]:
df_product %>% mutate(new_unit_price = floor(unit_cost / 0.7)) %>% mutate(profit_percentage = (new_unit_price - unit_cost)/new_unit_price) %>% head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,profit_percentage
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,P040101001,04,0401,040101,198,149,212,0.2971698
2,P040101002,04,0401,040101,218,164,234,0.2991453
3,P040101003,04,0401,040101,230,173,247,0.2995951
4,P040101004,04,0401,040101,248,186,265,0.2981132
5,P040101005,04,0401,040101,268,201,287,0.2996516
6,P040101006,04,0401,040101,298,224,320,0.3000000
7,P040101007,04,0401,040101,338,254,362,0.2983425
8,P040101008,04,0401,040101,420,315,450,0.3000000
9,P040101009,04,0401,040101,498,374,534,0.2996255


---
> R-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [177]:
df_product %>% mutate(new_unit_price = round(unit_cost / 0.7)) %>% mutate(profit_percentage = (new_unit_price - unit_cost)/new_unit_price) %>% head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,profit_percentage
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,P040101001,04,0401,040101,198,149,213,0.3004695
2,P040101002,04,0401,040101,218,164,234,0.2991453
3,P040101003,04,0401,040101,230,173,247,0.2995951
4,P040101004,04,0401,040101,248,186,266,0.3007519
5,P040101005,04,0401,040101,268,201,287,0.2996516
6,P040101006,04,0401,040101,298,224,320,0.3000000
7,P040101007,04,0401,040101,338,254,363,0.3002755
8,P040101008,04,0401,040101,420,315,450,0.3000000
9,P040101009,04,0401,040101,498,374,534,0.2996255


---
> R-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [179]:
df_product %>% mutate(new_unit_price = ceiling(unit_cost / 0.7)) %>% mutate(profit_percentage = (new_unit_price - unit_cost)/new_unit_price) %>% head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,profit_percentage
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,P040101001,04,0401,040101,198,149,213,0.3004695
2,P040101002,04,0401,040101,218,164,235,0.3021277
3,P040101003,04,0401,040101,230,173,248,0.3024194
4,P040101004,04,0401,040101,248,186,266,0.3007519
5,P040101005,04,0401,040101,268,201,288,0.3020833
6,P040101006,04,0401,040101,298,224,320,0.3000000
7,P040101007,04,0401,040101,338,254,363,0.3002755
8,P040101008,04,0401,040101,420,315,451,0.3015521
9,P040101009,04,0401,040101,498,374,535,0.3009346


---
> R-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [69]:
df_product %>% mutate(price_with_tax = floor(unit_price * 1.1)) %>% head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,price_with_tax
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>
1,P040101001,04,0401,040101,198,149,217
2,P040101002,04,0401,040101,218,164,239
3,P040101003,04,0401,040101,230,173,253
4,P040101004,04,0401,040101,248,186,272
5,P040101005,04,0401,040101,268,201,294
6,P040101006,04,0401,040101,298,224,327
7,P040101007,04,0401,040101,338,254,371
8,P040101008,04,0401,040101,420,315,462
9,P040101009,04,0401,040101,498,374,547


---
> R-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [70]:
left_join(df_receipt, df_product, by = "product_cd") %>%
    group_by(customer_id) %>%
    mutate(amount07 = amount * (category_major_cd == "07")) %>%
    summarize(Sum = sum(amount), Sum07 = sum(amount07)) %>%
    filter(Sum07 != 0) %>%
    mutate(percentage07 = Sum07 / Sum) %>%
    head(10)

customer_id,Sum,Sum07,percentage07
<chr>,<int>,<int>,<dbl>
CS001113000004,1298,1298,1.0000000
CS001114000005,626,486,0.7763578
CS001115000010,3044,2694,0.8850197
CS001205000004,1988,346,0.1740443
CS001205000006,3337,2004,0.6005394
CS001212000027,448,200,0.4464286
CS001212000031,296,296,1.0000000
CS001212000046,228,108,0.4736842
CS001212000070,456,308,0.6754386


---
> R-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [181]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    mutate(since_applied = difftime(ymd(sales_ymd), ymd(application_date))) %>%
    select(customer_id, sales_ymd, application_date, since_applied) %>%
    drop_na(everything()) %>%
    head(10)

,customer_id,sales_ymd,application_date,since_applied
,<chr>,<int>,<chr>,<drtn>
1,CS006214000001,20181103,20150201,1371 days
2,CS008415000097,20181118,20150322,1337 days
3,CS028414000014,20170712,20150711,732 days
4,CS025415000050,20180821,20160131,933 days
5,CS003515000195,20190605,20150306,1552 days
6,CS024514000042,20181205,20151010,1152 days
7,CS040415000178,20190922,20150627,1548 days
8,CS027514000015,20191010,20151101,1439 days
9,CS025415000134,20190918,20150720,1521 days


---
## R-071 PASS
> R-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [185]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    select(sales_ymd, application_date, customer_id) %>%
    mutate(since_applied = trunc(time_length(interval(strptime(application_date, "%Y%m%d"), strptime(as.character(sales_ymd), "%Y%m%d")), "month"))) %>%           
    head(10)
# PASS

,sales_ymd,application_date,customer_id,since_applied
,<int>,<chr>,<chr>,<dbl>
1,20181103,20150201,CS006214000001,45
2,20181118,20150322,CS008415000097,43
3,20170712,20150711,CS028414000014,24
4,20180821,20160131,CS025415000050,30
5,20190605,20150306,CS003515000195,50
6,20181205,20151010,CS024514000042,37
7,20190922,20150627,CS040415000178,50
8,20191010,20151101,CS027514000015,47
9,20190918,20150720,CS025415000134,49


---
> R-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [186]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    mutate(since_applied = floor(time_length(difftime(ymd(sales_ymd), ymd(application_date)), "year"))) %>%
    select(customer_id, sales_ymd, application_date, since_applied) %>%
    drop_na(everything()) %>%
    head(10)

,customer_id,sales_ymd,application_date,since_applied
,<chr>,<int>,<chr>,<dbl>
1,CS006214000001,20181103,20150201,3
2,CS008415000097,20181118,20150322,3
3,CS028414000014,20170712,20150711,2
4,CS025415000050,20180821,20160131,2
5,CS003515000195,20190605,20150306,4
6,CS024514000042,20181205,20151010,3
7,CS040415000178,20190922,20150627,4
8,CS027514000015,20191010,20151101,3
9,CS025415000134,20190918,20150720,4


---
> R-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [188]:
left_join(df_receipt, df_customer, by="customer_id") %>%
    mutate(since_applied = int_length(interval(ymd(application_date), ymd(sales_ymd)))) %>%
    select(customer_id, sales_ymd, application_date, since_applied) %>%
    drop_na(everything()) %>%
    head(10)

,customer_id,sales_ymd,application_date,since_applied
,<chr>,<int>,<chr>,<dbl>
1,CS006214000001,20181103,20150201,118454400
2,CS008415000097,20181118,20150322,115516800
3,CS028414000014,20170712,20150711,63244800
4,CS025415000050,20180821,20160131,80611200
5,CS003515000195,20190605,20150306,134092800
6,CS024514000042,20181205,20151010,99532800
7,CS040415000178,20190922,20150627,133747200
8,CS027514000015,20191010,20151101,124329600
9,CS025415000134,20190918,20150720,131414400


---
> R-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [75]:
df_receipt %>%
    mutate(since_Monday = wday(ymd(sales_ymd)+ 5) %% 7) %>%
    mutate(last_Monday = ymd(sales_ymd) - since_Monday) %>%
    select(sales_ymd, last_Monday, since_Monday) %>%
    head(10)

,sales_ymd,last_Monday,since_Monday
,<int>,<date>,<dbl>
1,20181103,2018-10-29,5
2,20181118,2018-11-12,6
3,20170712,2017-07-10,2
4,20190205,2019-02-04,1
5,20180821,2018-08-20,1
6,20190605,2019-06-03,2
7,20181205,2018-12-03,2
8,20190922,2019-09-16,6
9,20170504,2017-05-01,3


---
> R-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [76]:
sample_frac(tbl = df_customer, size = 0.01) %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS021513000218,石倉 美菜,1,女性,1966-12-04,52,259-1145,神奈川県伊勢原市板戸**********,S14021,20151028,A-20080224-3
2,CS005703000018,皆川 ヒロ,9,不明,1939-11-27,79,176-0024,東京都練馬区中村**********,S13005,20170417,0-00000000-0
3,CS024702000012,川越 龍吉,0,男性,1945-03-14,74,216-0031,神奈川県川崎市宮前区神木本町**********,S14024,20150324,0-00000000-0
4,CS003313000256,伴 知世,1,女性,1981-01-13,38,201-0012,東京都狛江市中和泉**********,S13003,20170123,0-00000000-0
5,CS011613000061,相原 崇史,9,不明,1953-10-10,65,223-0064,神奈川県横浜市港北区下田町**********,S14011,20150214,0-00000000-0
6,CS023515000039,三輪 夏希,1,女性,1967-07-05,51,212-0012,神奈川県川崎市幸区中幸町**********,S14023,20150203,D-20100127-E
7,CS023514000041,吹越 南朋,1,女性,1960-05-07,58,212-0012,神奈川県川崎市幸区中幸町**********,S14023,20150519,5-20081228-6
8,CS026615000124,古川 咲,1,女性,1955-12-29,63,251-0045,神奈川県藤沢市辻堂東海岸**********,S14026,20150412,4-20080128-5
9,CS019504000014,井上 憲史,0,男性,1965-08-14,53,176-0004,東京都練馬区小竹町**********,S13019,20141212,1-20081202-3


---
> R-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [77]:
df_customer %>% group_by(gender_cd) %>% mutate(num_rows = n()) %>% sample_frac(0.1, weight=num_rows) %>% count()

gender_cd,n
<chr>,<int>
0,298
1,1792
9,107


---
> R-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [195]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount)) %>%
    mutate(sum_amount_log = log(sum_amount + 0.5)) %>%
    filter(abs(sum_amount_log - mean(sum_amount_log)) / sd(sum_amount_log) > 3) %>%
    head(10)

customer_id,sum_amount,sum_amount_log
<chr>,<int>,<dbl>
ZZ000000000000,12395003,16.3328


---
> R-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [79]:
quantiles <- df_receipt %>%
    filter(!str_detect(customer_id, "^Z")) %>%
    group_by(customer_id) %>%
    summarize(Sum = sum(amount)) %>%
    summarize(quantiles = quantile(Sum)) %>%
    pull()
iqr <- quantiles["75%"] - quantiles["25%"]
df_receipt %>% filter(!str_detect(customer_id, "^Z")) %>% group_by(customer_id) %>% summarise(sum = sum(amount)) %>% filter(sum < quantiles["25%"] - 1.5 * iqr | quantiles["75%"] + 1.5 * iqr < sum) %>% head(10)

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.”


customer_id,sum
<chr>,<int>
CS001414000048,8584
CS001605000009,18925
CS002415000594,9568
CS004414000181,9584
CS005415000137,8734
CS006414000001,9156
CS006414000029,9179
CS006415000105,10042
CS006415000147,12723


---
> R-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [196]:
sapply(df_product, function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 7                  7

---
> R-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [81]:
df_product %>% count() %>% pull()
df_product %>% drop_na(everything()) %>% count() %>% pull()

[1] 10030

[1] 10023

---
> R-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [197]:
df_product %>% replace_na(replace = list(unit_price = floor(mean(.$unit_price, na.rm = TRUE)), unit_cost = floor(mean(.$unit_cost, na.rm = TRUE)))) %>% sapply(function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
> R-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [198]:
df_product %>% replace_na(replace = list(unit_price = median(.$unit_price, na.rm = TRUE), unit_cost = median(.$unit_cost, na.rm = TRUE))) %>% sapply(function(x) sum(is.na(x)))

product_cd  category_major_cd category_medium_cd  category_small_cd 
                 0                  0                  0                  0 
        unit_price          unit_cost 
                 0                  0

---
## R-083 PASS
> R-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [204]:
df_product %>%
    group_by(category_small_cd) %>%
    summarise(price_median = round(median(unit_price, na.rm = TRUE)),
              cost_median = round(median(unit_cost, na.rm = TRUE))) %>%
    inner_join(df_product, by = "category_small_cd") %>%
    mutate(unit_price = ifelse(is.na(unit_price), price_median, unit_price),
           unit_cost = ifelse(is.na(unit_cost), cost_median, unit_cost)) %>%
    sapply(function(x) sum(is.na(x)))

category_small_cd       price_median        cost_median         product_cd 
                 0                  0                  0                  0 
 category_major_cd category_medium_cd         unit_price          unit_cost 
                 0                  0                  0                  0

---
> R-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [216]:
left_join(df_customer, df_receipt, by = "customer_id") %>%
    group_by(customer_id) %>%
    mutate(amount_2019 = ifelse(str_detect(sales_ymd, "^2019"), amount, 0)) %>%
    summarise(prop_2019 = sum(amount_2019) / sum(amount)) %>%
    replace_na(replace = list(prop_2019 = 0)) %>%
    filter(prop_2019 > 0) -> df_r084

head(df_r084, 10)

sapply(df_r084, function(x) sum(is.na(x)))



customer_id,prop_2019
<chr>,<dbl>
CS001113000004,1.0000000
CS001114000005,0.3003195
CS001115000010,0.1898817
CS001205000004,0.3531187
CS001205000006,0.1456398
CS001211000025,1.0000000
CS001212000070,1.0000000
CS001214000009,0.1417289
CS001214000017,0.7168441


customer_id   prop_2019 
          0           0

---
> R-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [86]:
df_geocode_average <- df_geocode %>% group_by(postal_cd) %>% summarise(longitude = mean(longitude), latitude = mean(latitude))
df_r085 <- left_join(df_customer, df_geocode_average, by = "postal_cd")
df_r085 %>% head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.3178,35.41358
2,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.8350,35.67193
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.6897,35.67374
4,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.4836,35.39125
5,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.7078,35.54084
6,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0,139.6724,35.77073
7,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,139.8360,35.67818
8,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0,139.9047,35.65422
9,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0,139.8360,35.67818


---
> R-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [218]:
distance <- function(phi_1, phi_2, lambda_1, lambda_2) {
    a <- 6371 * acos(sin(phi_1*pi/180) * sin(phi_2*pi/180) + cos(phi_1*pi/180) * cos(phi_2*pi/180) * cos(lambda_2*pi/180 - lambda_1*pi/180))
    return(a)
    }

inner_join(df_r085, df_store, by = c("application_store_cd" = "store_cd")) %>%
    mutate(dist = distance(latitude.x, latitude.y, longitude.x, longitude.y)) %>%
    select(customer_id, address.x, address.y, dist) %>%
    head(10)

,customer_id,address.x,address.y,dist
,<chr>,<chr>,<chr>,<dbl>
1,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.3944087
2,CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.4511822
3,CS031415000172,東京都渋谷区代々木**********,東京都渋谷区初台二丁目,0.4117335
4,CS028811000001,神奈川県横浜市泉区和泉町**********,神奈川県横浜市瀬谷区二ツ橋町,8.0651960
5,CS001215000145,東京都大田区仲六郷**********,東京都大田区仲六郷二丁目,1.2684210
6,CS020401000016,東京都板橋区若木**********,東京都北区十条仲原三丁目,4.1859046
7,CS015414000103,東京都江東区北砂**********,東京都江東区南砂二丁目,1.4496734
8,CS029403000008,千葉県浦安市海楽**********,千葉県浦安市東野一丁目,0.8048581
9,CS015804000004,東京都江東区北砂**********,東京都江東区南砂二丁目,1.4496734


---
> R-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [88]:
df_customer %>% nrow()
df_r087 <- left_join(df_customer, df_receipt, by = "customer_id") %>% group_by(customer_id) %>% summarise(sum = sum(amount), customer_name, postal_cd) %>% arrange(desc(sum), customer_id) %>% ungroup() %>% distinct(customer_name, postal_cd, .keep_all=TRUE)
df_r087 %>% nrow()

[1] 21971

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.”
`summarise()` has grouped output by 'customer_id'. You can override using the
`.groups` argument.


[1] 21941

---
> R-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [89]:
left_join(df_r087, df_customer, by = "customer_id") %>% count()

n
<int>
21941


---
> R-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [90]:
train <- df_r087 %>% sample_frac(size = 0.8, replace = FALSE)
test <- df_r087 %>% anti_join(train, by = c("customer_id"))

train %>% count()
test %>% count()

n
<int>
17553


n
<int>
4388


---
## R-090 PASS
> R-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [219]:
#caretのcreateTimeSlicesを使った例
df_ts_amount <- df_receipt %>%
    group_by(substr(as.character(sales_ymd), 1, 6)) %>%
    summarise(sum_amount = sum(amount), .groups = "drop")

colnames(df_ts_amount) <- c("sales_ym", "sales_amount")

timeSlice <- createTimeSlices(df_ts_amount$sales_ym, 
                              initialWindow = 12, 
                              horizon = 6, 
                              skip = 5, 
                              fixedWindow = TRUE)

df_train_1 <- df_ts_amount[timeSlice$train[[1]],]
df_train_2 <- df_ts_amount[timeSlice$train[[2]],]
df_train_3 <- df_ts_amount[timeSlice$train[[3]],]

df_test_1 <- df_ts_amount[timeSlice$test[[1]],]
df_test_2 <- df_ts_amount[timeSlice$test[[2]],]
df_test_3 <- df_ts_amount[timeSlice$test[[3]],]

---
## R-091 PASS
> R-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [220]:
#recipesパッケージを使った例
df_sales_amount <- df_receipt %>%
    group_by(customer_id) %>%
    summarise(sum_amount = sum(amount), .groups = "drop") %>%
    right_join(df_customer, by = "customer_id") %>%
    mutate(is_buy_flag = factor(ifelse(is.na(sum_amount), 0, 1)))

df_down_sampling <- df_sales_amount %>% 
    recipe() %>% 
    step_downsample(is_buy_flag, seed = 71) %>%
    prep() %>% 
    juice()

df_down_sampling %>%
    group_by(is_buy_flag) %>%
    summarise(cnt = n(), .groups = "drop")

is_buy_flag,cnt
<fct>,<int>
0,8306
1,8306


---
## R-092 PASS
> R-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [221]:
df_gender_std = unique(df_customer[c("gender_cd", "gender")])

df_customer_std = df_customer[, colnames(df_customer) != "gender"]

---
> R-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [97]:
df_r093 <- left_join(df_product, df_category, by = "category_small_cd")
df_r093 %>% head()

,product_cd,category_major_cd.x,category_medium_cd.x,category_small_cd,unit_price,unit_cost,category_major_cd.y,category_major_name,category_medium_cd.y,category_medium_name,category_small_name
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,P040101001,04,0401,040101,198,149,04,惣菜,0401,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,04,惣菜,0401,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,04,惣菜,0401,御飯類,弁当類
4,P040101004,04,0401,040101,248,186,04,惣菜,0401,御飯類,弁当類
5,P040101005,04,0401,040101,268,201,04,惣菜,0401,御飯類,弁当類
6,P040101006,04,0401,040101,298,224,04,惣菜,0401,御飯類,弁当類


---
> R-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [101]:
write.csv(df_r093, "./data/.product_R093.csv", row.names=FALSE)

---
> R-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [99]:
write.csv(df_r093, "./data/.product_R093_cp932.csv", row.names=FALSE, fileEncoding = "CP932")

---
> R-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [100]:
write.table(df_r093, "./data/.product_R093_no_header.csv", row.names = FALSE, col.names = FALSE, sep = ",")

---
> R-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [103]:
read.csv("./data/.product_R093.csv", fileEncoding = "utf8") %>% head(3)

,product_cd,category_major_cd.x,category_medium_cd.x,category_small_cd,unit_price,unit_cost,category_major_cd.y,category_major_name,category_medium_cd.y,category_medium_name,category_small_name
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>
1,P040101001,4,401,40101,198,149,4,惣菜,401,御飯類,弁当類
2,P040101002,4,401,40101,218,164,4,惣菜,401,御飯類,弁当類
3,P040101003,4,401,40101,230,173,4,惣菜,401,御飯類,弁当類


---
> R-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [105]:
read.csv("./data/.product_R093_no_header.csv", fileEncoding = "utf8", header = FALSE) %>% head(3)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>
1,P040101001,4,401,40101,198,149,4,惣菜,401,御飯類,弁当類
2,P040101002,4,401,40101,218,164,4,惣菜,401,御飯類,弁当類
3,P040101003,4,401,40101,230,173,4,惣菜,401,御飯類,弁当類


---
> R-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [109]:
write.table(df_r093, "./data/.product_R093.tsv", row.names = FALSE, sep = "\t")

---
> R-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [113]:
read.table("./data/.product_R093.tsv", fileEncoding = "utf8", sep = "\t", header = TRUE) %>% head(3)

,product_cd,category_major_cd.x,category_medium_cd.x,category_small_cd,unit_price,unit_cost,category_major_cd.y,category_major_name,category_medium_cd.y,category_medium_name,category_small_name
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>
1,P040101001,4,401,40101,198,149,4,惣菜,401,御飯類,弁当類
2,P040101002,4,401,40101,218,164,4,惣菜,401,御飯類,弁当類
3,P040101003,4,401,40101,230,173,4,惣菜,401,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！